In [1]:
from keras.datasets import cifar10
from keras.applications import InceptionResNetV2
from keras.utils import np_utils
from keras.callbacks import TensorBoard
from keras.layers import Input, Dense, Conv2D, MaxPool2D, Concatenate, Flatten, GlobalAveragePooling2D, Dropout
from keras.models import Model, load_model
from keras.engine.topology import get_source_inputs

import matplotlib.pyplot as plt
from time import time


C:\Users\michael\Anaconda3\envs\deep\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Load Cifar10 data

In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')/255
x_test = x_test.astype('float32')/255
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

## Network definition
The network has a stem at the start, followed by multiple inception blocks.
After the inception blocks a globalaverage is applied followed by a dense output layer.

In [3]:
def create_stem():
    img_input = Input(shape=(32, 32, 3))
    stem = Conv2D(32, 3, strides=2, padding="same", name="stem-conv-1")(img_input)
    stem = Conv2D(32, 3, strides=1, padding="same", name="stem-conv-2")(stem)
    stem = Conv2D(64, 3, strides=1, padding="same", name="stem-conv-3")(stem)
    stem = MaxPool2D(3, strides=2)(stem)
    stem = Conv2D(80, 1, padding="same", name="stem-conv-4")(stem)
    stem = Conv2D(192, 3, padding="same", name="stem-conv-5")(stem)
    stem = MaxPool2D(3, strides=2, name="stem-maxpool2")(stem)
    return stem, get_source_inputs(img_input)


def get_flower(inceptions):
    flower = GlobalAveragePooling2D(name="flower-globalaverage")(inceptions)
    flower = Dropout(0.2)(flower)
    flower = Dense(10, activation="softmax")(flower)
    return flower
    
def add_inception(previous, count=1):
    
    # 1x1
    branch1x1 = Conv2D(64, (1, 1), padding="same", activation="relu", name="inception-{}-1x1".format(count))(previous)
    
    # 3x3
    branch3x3 = Conv2D(64, (1, 1), padding="same", activation="relu", name="inception-{}-3x3-1".format(count))(previous)
    branch3x3 = Conv2D(64, (3, 3), padding="same", activation="relu", name="inception-{}-3x3-2".format(count))(branch3x3)
    
    # 5x5
    branch5x5 = Conv2D(48, (1, 1), padding="same", activation="relu", name="inception-{}-5x5-1".format(count))(previous)
    branch5x5 = Conv2D(64, (5, 5), padding="same", activation="relu", name="inception-{}-5x5-2".format(count))(branch5x5)
    
    # 3x3 pooling
    branch_pool = MaxPool2D((3, 3), strides=(1, 1), padding="same", name="inception-{}-pool-1".format(count))(previous)
    branch_pool = Conv2D(64, (1, 1), padding="same", activation="relu", name="inception-{}-pool-2".format(count))(branch_pool)
    
    next = Concatenate(axis=3, name="inception-{}-concat".format(count))([branch1x1, branch3x3, branch5x5, branch_pool])
    return next

In [6]:
stem, img_input = create_stem()
output = add_inception(stem, count=1)
# for i in range(2, 2):
#     output = add_inception(output, count=i)
flower = get_flower(output)

model = Model(img_input, flower, name='inception_gen')
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
stem-conv-1 (Conv2D)            (None, 16, 16, 32)   896         input_3[0][0]                    
__________________________________________________________________________________________________
stem-conv-2 (Conv2D)            (None, 16, 16, 32)   9248        stem-conv-1[0][0]                
__________________________________________________________________________________________________
stem-conv-3 (Conv2D)            (None, 16, 16, 64)   18496       stem-conv-2[0][0]                
__________________________________________________________________________________________________
max_poolin

## Training
The model will be saved afterwards (and overwritten) as `model.h5`. A tensorboard log will be written. The tensorboard can be started with ```tensorboard --logdir=c:/tmp/logs/``` and visited at [http://michael-laptop:6006](http://michael-laptop:6006/#scalars&tagFilter=acc&regexInput=cifar)

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from time import strftime

model_path = "model_checkpoint.h5"
run_name = "cifar10-{}".format(strftime('%a-%H.%M'))
print("Starting run {} ".format(run_name))

cbs = [
    TensorBoard(log_dir="../logs/{}".format(run_name)),
    EarlyStopping(monitor="val_acc", patience=20, verbose=1),
    ModelCheckpoint(model_path, monitor="val_acc", save_best_only=True, save_weights_only=False, verbose=1)
]

model.fit(x_train, y_train, epochs=100, batch_size=32, validation_data=[x_test, y_test], callbacks=cbs)
model.save("model.h5")

Starting run cifar10-Wed-16.12 
Train on 50000 samples, validate on 10000 samples
Epoch 1/100
50000/50000 [==============================] - 18s 357us/step - loss: 1.4116 - acc: 0.4897 - val_loss: 1.0157 - val_acc: 0.6402

Epoch 00001: val_acc improved from -inf to 0.64020, saving model to model_checkpoint.h5
Epoch 2/100
50000/50000 [==============================] - 17s 337us/step - loss: 0.9523 - acc: 0.6652 - val_loss: 0.8735 - val_acc: 0.6964

Epoch 00002: val_acc improved from 0.64020 to 0.69640, saving model to model_checkpoint.h5
Epoch 3/100
50000/50000 [==============================] - 17s 340us/step - loss: 0.7925 - acc: 0.7240 - val_loss: 0.8450 - val_acc: 0.7071

Epoch 00003: val_acc improved from 0.69640 to 0.70710, saving model to model_checkpoint.h5
Epoch 4/100
50000/50000 [==============================] - 17s 343us/step - loss: 0.6967 - acc: 0.7561 - val_loss: 0.7366 - val_acc: 0.7469

Epoch 00004: val_acc improved from 0.70710 to 0.74690, saving model to model_checkpo

## Model persistence

In [ ]:
model.save("custom.h5")

In [ ]:
model = load_model("model.h5")

## Validation